In [1]:
import torch
from torch import nn

# 一、填充(padding)
"""
在应用多层卷积时，常常丢失边缘像素。由于通常使用小卷积核，因此对于任何单个卷积，
可能只会丢失几个像素。但随着应用许多连续卷积层，累积丢失的像素数就多了。
解决这个问题的简单方法即为填充:在输入图像的边界填充元素(通常填充元素是0)
"""
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])
# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)